In [ ]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-07
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-07
@Title : DynamnoDB BOTO3 programs

'''

In [1]:
import boto3

In [2]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')

### 1. CRUD operations on DynamoDB tables using BOTO3

#### 1.a. C: CREATE

In [7]:
def connect_to_dynamoDB():
    """

    Description:
        To connect to AWS DynamoDB service.
    Parameter:
        No parameters
    Return:
        ServiceResource dyDB

    """
    dyDB =  boto3.resource('dynamodb')
    return dyDB

def create_items(table,employee_details):
    """

    Description:
        To create items in a table of DynamoDB.
    Parameter:
        dynamodb.table table
    Return:
        No values returned.

    """
    table.put_item(
        Item={
                'EMPID': employee_details[0],
                'DateOfJoin': employee_details[1],
                'FirstName': employee_details[2],
                'LastName': employee_details[3],
                'Role': employee_details[4],
                'Department': employee_details[5],
                'ContactNo': employee_details[6]
            }
    )

def main():

    dyDB = connect_to_dynamoDB()

    # Create the DynamoDB table.
    table = dyDB.create_table(
        TableName='EmployeeBOTO3',
        KeySchema=[
            {
                'AttributeName': 'EMPID',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'DateOfJoin',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'EMPID',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'DateOfJoin',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    # Wait until the table exists.
    table.wait_until_exists()

    # Print out some data about the table.
    print(f"No. of items in the table :{table.item_count}")
    print(f"DateTime creation of Table : {table.creation_date_time}")

    # table = dyDB.Table('EmployeeBOTO3')

    # Adding Items to the table
    employee_1 = ["EMP001","20-10-2015","John","Smith","Data Engineer","IT","9876543210"]
    create_items(table,employee_1)
    employee_2 = ["EMP002","19-04-2019","Rahul","Kumar","Backend Developer","IT","9102837465"]
    create_items(table,employee_2)
    employee_3 = ["EMP003","13-07-2011","Rahul","Kumar","Business Development","CEO","8555090324"]
    create_items(table,employee_3)
    employee_4 = ["EMP004","10-04-2019","Vighnesh","Bilgi","Data Analyst","IT","9049480396"]
    create_items(table,employee_4)

if __name__ == "__main__":
    main()